In [2]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""
import sys

In [3]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from databasetools.mongo import *
from newstools.goodarticle.utils import *
from nlptools.preprocessing import *
from nlptools.news import parser as newsParser
from machinelearning.iterator import *
from twinews.utils import *
from twinews.models.ranking import *
from twinews.evaluation.utils import *

In [4]:
from nlptools.basics import *
from nltk.stem import WordNetLemmatizer 
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import LdaMulticore
from gensim.test.utils import common_corpus, common_dictionary
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
import gensim
from math import log2
from math import sqrt
from numpy import asarray

In [5]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from twinews.yfnotebooks.load_data import load_char_data,char_index  # load data tools
from twinews.yfnotebooks.convnet import args

Instructions for updating:
non-resource variables are not supported in the long term


In [6]:
isNotebook = '__file__' not in locals()
TEST = isNotebook

In [7]:
logger = Logger(tmpDir('logs') + "/Convnet.log") if isNotebook else Logger("Convnet-" + getHostname() + ".log")
tt = TicToc(logger=logger)
tt.tic()

--> tictoc starts...


-1

In [8]:
modelName = "Convnet"

# config

In [14]:
config = \
{
    'splitVersion': 2,
    'testVersion':2, # with some parameters changed    
    
    # for input data
    'maxUsers': 5 if TEST else None, # Sub-sampling
    'maxDocuments': None,
    'useExtraNews': 0 if TEST else None, # None = unlimited, 0 = no extra news
    'minDF': 1 / 2000, # Remove words that have a document frequency ratio lower than 1 / 2000
    'maxDF': 20, # Remove top 20 voc elements
    'lowercase': True,
    'doLemmatization': True,
    

    
    # for ouput
    'similarity': 'cosine',

#     'historyRef': 0.3, # 1, 1.0, 0.5, 0.3, 3, 10
}

# graph 

In [10]:
#-*-coding:utf-8-*-
# defining the training graph

class Graph:

    def __init__(self):
        self.p = tf.placeholder(dtype=tf.int32, shape=(None, args.seq_length), name='p')
        self.h = tf.placeholder(dtype=tf.int32, shape=(None, args.seq_length), name='h')
        self.y = tf.placeholder(dtype=tf.int32, shape=None, name='y')
        self.keep_prob = tf.placeholder(dtype=tf.float32, name='drop_rate')

        self.embedding = tf.get_variable(dtype=tf.float32, shape=(args.vocab_size, args.char_embedding_size),
                                         name='embedding')
        self.M = tf.Variable(tf.random_normal(shape=(args.n_filter, args.n_filter), mean=0, stddev=1),
                             name='M',
                             dtype=tf.float32)
        self.x_feat = tf.Variable(tf.random_normal(shape=(args.batch_size, args.n_filter), mean=0, stddev=1),
                                  name='x_feat',
                                  dtype=tf.float32)

        self.forward()

    def dropout(self, x):
        return tf.nn.dropout(x, keep_prob=self.keep_prob)

    def forward(self):
        p_embedding = tf.nn.embedding_lookup(self.embedding, self.p)
        h_embedding = tf.nn.embedding_lookup(self.embedding, self.h)

        p_embedding = tf.expand_dims(p_embedding, axis=3)
        h_embedding = tf.expand_dims(h_embedding, axis=3)
        p = tf.layers.conv2d(p_embedding,
                             filters=args.n_filter,
                             kernel_size=(args.filter_width, args.filter_height),
                             activation='relu')
        h = tf.layers.conv2d(h_embedding,
                             filters=args.n_filter,
                             kernel_size=(args.filter_width, args.filter_height),
                             activation='relu')

        pool_width = args.seq_length + 1 - args.filter_width
        p_max = tf.layers.max_pooling2d(p, pool_size=(pool_width, 1), strides=1)
        h_max = tf.layers.max_pooling2d(h, pool_size=(pool_width, 1), strides=1)

        p_max = tf.squeeze(tf.squeeze(p_max, axis=1), axis=1)
        h_max = tf.squeeze(tf.squeeze(h_max, axis=1), axis=1)

        # sim = tf.matmul(tf.matmul(p_max, self.M), tf.transpose(h_max))
        sim = tf.matmul(p_max, self.M)
        sim = tf.reduce_sum(tf.multiply(sim, h_max), axis=1, keep_dims=True)

        x = tf.concat((p_max, h_max, sim), axis=1)

        v = tf.layers.dense(x, 256, activation='relu')
        v = self.dropout(v)
        logits = tf.layers.dense(v, 2, activation='relu')

        self.train(logits)

    def train(self, logits):
        y = tf.one_hot(self.y, args.class_size)
        loss = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits)
        self.loss = tf.reduce_mean(loss)
        self.train_op = tf.train.AdamOptimizer(args.learning_rate).minimize(self.loss)
        # use the logits output(second last) for ranking
        self.logits = logits
        prediction = tf.argmax(logits, axis=1)
        # use the 0/1 prediction for ranking
        self.prediction = prediction
        correct_prediction = tf.equal(tf.cast(prediction, tf.int32), self.y)
        self.acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


# training

In [32]:
# training

#from twinews.yfnotebooks.convnet import args

tf.reset_default_graph()

p, h, y = load_char_data('/home/yuting/PycharmProjects/data/dssm_title_train.csv', data_size=None)
p_eval, h_eval, y_eval = load_char_data('/home/yuting/PycharmProjects/data/dssm_title_dev.csv', data_size=args.batch_size)

p_holder = tf.placeholder(dtype=tf.int32, shape=(None, args.seq_length), name='p')
h_holder = tf.placeholder(dtype=tf.int32, shape=(None, args.seq_length), name='h')
y_holder = tf.placeholder(dtype=tf.int32, shape=None, name='y')

dataset = tf.data.Dataset.from_tensor_slices((p_holder, h_holder, y_holder))
dataset = dataset.batch(args.batch_size).repeat(args.epochs)
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

model = Graph()
saver = tf.train.Saver()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.9

with tf.Session(config=config)as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(iterator.initializer, feed_dict={p_holder: p, h_holder: h, y_holder: y})
    steps = int(len(y) / args.batch_size)
    for epoch in range(args.epochs):
        for step in range(steps):
            p_batch, h_batch, y_batch = sess.run(next_element)
            _, loss, acc = sess.run([model.train_op, model.loss, model.acc],
                                    feed_dict={model.p: p_batch,
                                               model.h: h_batch,
                                               model.y: y_batch,
                                               model.keep_prob: args.keep_prob})
            print('epoch:', epoch, ' step:', step, ' loss: ', loss, ' acc:', acc)

        loss_eval, acc_eval = sess.run([model.loss, model.acc],
                                       feed_dict={model.p: p_eval,
                                                  model.h: h_eval,
                                                  model.y: y_eval,
                                                  model.keep_prob: 1})
        print('loss_eval: ', loss_eval, ' acc_eval:', acc_eval)
        print('\n')
        saver.save(sess, f'/home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_{epoch}.ckpt')


epoch: 0  step: 0  loss:  0.69322234  acc: 0.4765625
epoch: 0  step: 1  loss:  0.6932197  acc: 0.4970703
epoch: 0  step: 2  loss:  0.69299364  acc: 0.5126953
epoch: 0  step: 3  loss:  0.6931623  acc: 0.4951172
epoch: 0  step: 4  loss:  0.6929022  acc: 0.51171875
epoch: 0  step: 5  loss:  0.6932014  acc: 0.4736328
epoch: 0  step: 6  loss:  0.69260746  acc: 0.5332031
epoch: 0  step: 7  loss:  0.6925728  acc: 0.51464844
epoch: 0  step: 8  loss:  0.69272065  acc: 0.4873047
epoch: 0  step: 9  loss:  0.69246507  acc: 0.49121094
epoch: 0  step: 10  loss:  0.6926708  acc: 0.4794922
epoch: 0  step: 11  loss:  0.6919373  acc: 0.50683594
epoch: 0  step: 12  loss:  0.69185096  acc: 0.5107422
epoch: 0  step: 13  loss:  0.69146144  acc: 0.5078125
epoch: 0  step: 14  loss:  0.6918249  acc: 0.48535156
epoch: 0  step: 15  loss:  0.691381  acc: 0.49804688
epoch: 0  step: 16  loss:  0.69045544  acc: 0.5058594
epoch: 0  step: 17  loss:  0.6903516  acc: 0.49316406
epoch: 0  step: 18  loss:  0.6904422  acc:

epoch: 7  step: 16  loss:  0.12527652  acc: 0.9609375
epoch: 7  step: 17  loss:  0.115872696  acc: 0.95996094
epoch: 7  step: 18  loss:  0.11709474  acc: 0.95703125
loss_eval:  0.122704856  acc_eval: 0.9658203


epoch: 8  step: 0  loss:  0.11258467  acc: 0.96972656
epoch: 8  step: 1  loss:  0.1276662  acc: 0.9560547
epoch: 8  step: 2  loss:  0.13018908  acc: 0.9638672
epoch: 8  step: 3  loss:  0.128878  acc: 0.95703125
epoch: 8  step: 4  loss:  0.12023044  acc: 0.9589844
epoch: 8  step: 5  loss:  0.1286129  acc: 0.96191406
epoch: 8  step: 6  loss:  0.1048397  acc: 0.9658203
epoch: 8  step: 7  loss:  0.086310655  acc: 0.9758308
epoch: 8  step: 8  loss:  0.121877454  acc: 0.95703125
epoch: 8  step: 9  loss:  0.10741159  acc: 0.9658203
epoch: 8  step: 10  loss:  0.09866793  acc: 0.9658203
epoch: 8  step: 11  loss:  0.109577864  acc: 0.9658203
epoch: 8  step: 12  loss:  0.10319106  acc: 0.96972656
epoch: 8  step: 13  loss:  0.107962176  acc: 0.9716797
epoch: 8  step: 14  loss:  0.12253089 

# get data

In [17]:
# Getting users and news
evalData = getEvalData(config['splitVersion'], maxExtraNews=0,
                       maxUsers=config['maxUsers'])
(trainUsers, testUsers, trainNews, testNews, candidates, extraNews) = \
(evalData['trainUsers'], evalData['testUsers'], evalData['trainNews'],
 evalData['testNews'], evalData['candidates'], evalData['extraNews'])
bp(evalData.keys(), 5, logger)
log(b(evalData['meta'], 5), logger)

--> tic: 19.26s | message: Eval data loaded


{ candidates, extraNews, meta, testNews, testUsers, trainNews, trainUsers }
{ 'created': 2020.03.24-14.28.06, 'endDate': 2018-01-15, 'id': 2, 'ranksLength': 1000, 'splitDate': 2017-12-25, 'startDate': 2017-10-01, 'testMaxNewsPerUser': 97, 'testMeanNewsPerUser': 7.22, 'testMinNewsPerUser': 2, 'testNewsCount': 71781, 'totalNewsAvailable': 570210, 'trainMaxNewsPerUser': 379, 'trainMeanNewsPerUser': 26.48, 'trainMinNewsPerUser': 8, 'trainNewsCount': 237150, 'usersCount': 15905 }


--> toc total duration: 19.48s | message: Got Twinews evaluation data


In [18]:
trainNewsList = shuffle(list(trainNews), seed=0)
testNewsList = shuffle(list(testNews), seed=0)

In [19]:
newsList = trainNewsList + testNewsList
log(str(len(newsList)) + " urls for newsList:\n" + b(newsList), logger=logger)

5006 urls for newsList:
[
  http://www.dailymail.co.uk/news/article-5186005/Trump-removes-climate-change-national-security-polic,
  http://www.edweek.org/ew/articles/2017/11/29/when-it-comes-to-sexual-harassment-schools.html?cmp=soc,
  ...,
  https://patch.com/new-york/new-york-city/rikers-island-visitors-put-through-hell-see-loved-ones-repo,
  http://www.tabletmag.com/scroll/196615/four-lessons-in-virtue-from-martin-luther-king-jr
]


In [20]:
# get all the titles of news urls link to form (url: title) dict
titles = getNewsTitles(newsList)

twinews news (version 1.0) initialised.
  100% [====================] (total duration: 3m 32.419s, mean duration: 0.042s)


In [21]:
# tokenize the tiltles since there was no tokenized title in the database
titlesToken = []
for i in pb(list(range(len(titles))), logger=logger, message="tokenizing"):
    titlesToken.append(wordTokenize(titles[i]))
bp(titlesToken,2)

tokenizing   0% [                    ]
tokenizing   9% [=                   ] (4.235s left)
tokenizing  19% [===                 ] (2.243s left)
tokenizing  29% [=====               ] (1.495s left)
tokenizing  39% [=======             ] (1.172s left)
tokenizing  49% [=========           ] (0.852s left)
tokenizing  59% [===========         ] (0.615s left)
tokenizing  69% [=============       ] (0.425s left)
tokenizing  79% [===============     ] (0.266s left)
tokenizing  89% [=================   ] (0.125s left)
tokenizing  99% [=================== ] (0.001s left)
tokenizing 100% [====================] (total duration: 1.2s, mean duration: 0s)


[ [ Trump, removes, ..., Mail, Online ], [ When, It, ..., Education, Week ], ..., [ Rikers, Island, ..., NY, Patch ], [ Four, Lessons, ..., Tablet, Magazine ] ]


In [22]:
for i in pb(list(range(len(titlesToken))), logger=logger, message="Lower casing"):
    if titlesToken[i] == None:
        #print(titlesToken[i])  # just to see how many urls have empty title
        continue
    else:
        for u in range(len(titlesToken[i])):
            titlesToken[i][u] = titlesToken[i][u].lower()
bp(titlesToken, logger)

Lower casing   0% [                    ]
Lower casing   9% [=                   ] (0s left)
Lower casing  19% [===                 ] (0.04s left)
Lower casing  29% [=====               ] (0.046s left)
Lower casing  39% [=======             ] (0.045s left)
Lower casing  49% [=========           ] (0.04s left)
Lower casing  59% [===========         ] (0.033s left)
Lower casing  69% [=============       ] (0.025s left)
Lower casing  79% [===============     ] (0.017s left)
Lower casing  89% [=================   ] (0.008s left)
Lower casing  99% [=================== ] (0s left)
Lower casing 100% [====================] (total duration: 0.09s, mean duration: 0s)
[ [ trump, removes, ..., mail, online ], [ when, it, ..., education, week ], ..., [ rikers, island, ..., ny, patch ], [ four, lessons, ..., tablet, magazine ] ]


In [23]:
# lemmatization
lemmatizer = WordNetLemmatizer()
pbar = ProgressBar(len(titlesToken), logger=logger, message="Lemmatization")
for i in range(len(titlesToken)):
    if titlesToken[i] == None:
        # print(titlesToken[i])  # same as last one
        continue
    else:
        for u in range(len(titlesToken[i])):
            titlesToken[i][u] = lemmatizer.lemmatize(titlesToken[i][u])
    pbar.tic()
bp(titlesToken, logger)

Lemmatization   0% [                    ]
Lemmatization   9% [=                   ] (8.291s left)
Lemmatization  19% [===                 ] (3.805s left)
Lemmatization  29% [=====               ] (2.29s left)
Lemmatization  39% [=======             ] (1.518s left)
Lemmatization  49% [=========           ] (1.042s left)
Lemmatization  59% [===========         ] (0.715s left)
Lemmatization  69% [=============       ] (0.473s left)
Lemmatization  79% [===============     ] (0.284s left)
Lemmatization  89% [=================   ] (0.13s left)
Lemmatization  99% [=================== ] (0.001s left)
[ [ trump, remove, ..., mail, online ], [ when, it, ..., education, week ], ..., [ rikers, island, ..., ny, patch ], [ four, lesson, ..., tablet, magazine ] ]


In [24]:
# build a dict with url as key, and tokenized title as value
urlTitles= dict()
for i in range(len(titlesToken)):
    urlTitles[newsList[i]] = titlesToken[i]
bp(urlTitles, 2, logger)

{
  http://a.msn.com/01/en-ie/BBHzIAS?ocid=st: [ rare, supermoon, ..., see, it ],
  http://a.msn.com/01/en-us/AAuFzwx?ocid=st: [ gop, senator, ..., 's, remark ],
  http://a.msn.com/01/en-us/BBHs8ye?ocid=st: [ eric, garner, ..., :, report ],
  http://a.msn.com/02/en-us/BBI8v4T?ocid=st: [ angry, fan, ..., title, game ],
  http://a.msn.com/06/en-us/BBH7EiO?ocid=st: [ this, is, ..., any, restaurant ],
  ...,
  https://www.zerohedge.com/news/2018-01-04/trump-admin-bans-all-personal-devices-white-house: [ trump, admin, ..., zero, hedge ],
  https://www.zerohedge.com/news/2018-01-05/fbi-chief-foia-officer-every-single-memo-comey-leaked-was-classified: [ fbi, chief, ..., zero, hedge ],
  https://www.zerohedge.com/news/2018-01-05/new-video-emerges-isis-convoys-leaving-raqqa-under-us-coalition-watch: [ new, video, ..., zero, hedge ],
  https://www.zerohedge.com/news/2018-01-08/credit-card-debt-hits-all-time-high-consumers-unleash-historic-shopping-spree: [ credit, card, ..., zero, hedge ],
  htt

# load the vocab

In [25]:
# You need to change the file path, I put it under ../Twinews/twinews/yfnotebooks/title_vocab.txt
# to filter out the words not in the vacab to shrink down the size of the query
titleVocab = []
with open('/home/yuting/PycharmProjects/data/title_vocab.txt','r') as file:
    for line in file.readlines():
        titleVocab.append(line.strip('\n'))

bp(titleVocab,2)

[ !, #, ..., •, … ]


In [26]:
len(trainUsers.keys())

5

# build user history

In [27]:
# for multiple users
trainUserQuery = dict()
pbar = ProgressBar(len(trainUsers.keys()), logger=logger, message="build user history")

# for each user
for usr in trainUsers.keys():
    queryFinal = []
    
    # choose 15 urls in their history to form the query text
    if len(list(trainUsers[usr])) < 15:
        query = list(trainUsers[usr])
    else:
        query = random.sample(list(trainUsers[usr]),15)
        
    # put 15 titles together, using extend
    temp = []
    for url in query:
        if urlTitles[url] == None:
            continue
        else:
            temp.extend(urlTitles[url])
            
    # the following two steps are making the query titles more simplified and efficient        
    # filter what is not in the vocab
    tempCompact = []
    for i in range(len(temp)):
        if temp[i] in titleVocab:
            tempCompact.append(temp[i])
    
    # remove duplicated & detokenize
    queryCompact = detokenize(list(set(tempCompact)))
    
    queryFinal.append(queryCompact)
    trainUserQuery[usr] = queryFinal
    
    pbar.tic()

build user history  20% [====                ] (0.159s left)
build user history  40% [========            ] (0.134s left)
build user history  60% [============        ] (0.086s left)
build user history  80% [================    ] (0.042s left)
build user history 100% [====================] (total duration: 0.22s, mean duration: 0.044s)


# rankings

In [33]:
# I'm not sure whether you can use this checkpoint file
def computeSimilarity(p,h,y):
    
    tf.reset_default_graph() 
    model = Graph()
    saver = tf.train.Saver()
    
    # restore the trianed model
    with tf.Session()as sess:
        sess.run(tf.global_variables_initializer())
        saver.restore(sess, '/home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_9.ckpt')
        loss, acc, prediction = sess.run([model.loss, model.acc, model.prediction],
                             feed_dict={model.p: p,
                                        model.h: h,
                                        model.y: y,
                                        model.keep_prob: 1})

        #print('loss: ', loss, ' acc:', acc)
        return prediction
#logit = computeSimilarity(p,h,y)

In [35]:
# rankings of the candidates with logits(second last uotput) results
pbar = ProgressBar(len(trainUsers.keys()), logger=logger, message="testing")
rankings = dict()
for usr in trainUsers.keys():
    usrRankings = []
    
    # get historical data as query(p)
    query = trainUserQuery[usr] * 1000
    
    for candidates in evalData['candidates'][usr]:
        candidates = list(candidates)
        # get 1000 h as a list(h)
        h_temp = []
        for url in candidates:
            h_temp.append(detokenize(urlTitles[url]))
        # get p,h
        p,h = char_index(query,h_temp)
        y = np.zeros((1000))                    # whatever initialization
        
        scoresCandidates = []
        logit = computeSimilarity(p,h,y)            # get the 1000 scores list
        scoresCandidates = zip(candidates,list(logit[:,1]))
        ranking = sortBy(scoresCandidates, index=1, desc=True)
        ranking = [e[0] for e in ranking]
        usrRankings.append(ranking)
    rankings[usr] = usrRankings
    
    pbar.tic()

INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_9.ckpt


IndexError: too many indices for array

In [36]:
# rankings of the candidates with 0/1 classification results
pbar = ProgressBar(len(trainUsers.keys()), logger=logger, message="testing")
rankings = dict()
for usr in trainUsers.keys():
    usrRankings = []
    
    # get historical data as query(p)
    query = trainUserQuery[usr] * 1000
    
    for candidates in evalData['candidates'][usr]:
        candidates = list(candidates)
        # get 1000 h as a list(h)
        h_temp = []
        for url in candidates:
            h_temp.append(detokenize(urlTitles[url]))
        # get p,h
        p,h = char_index(query,h_temp)
        y = np.zeros((1000))                    # whatever initialization
        
        scoresCandidates = []
        prediction = computeSimilarity(p,h,y)            # get the 1000 scores list
        scoresCandidates = zip(candidates,prediction)
        ranking = sortBy(scoresCandidates, index=1, desc=True)
        ranking = [e[0] for e in ranking]
        usrRankings.append(ranking)
    rankings[usr] = usrRankings
    
    pbar.tic()

INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_9.ckpt


testing  20% [====                ] (2m 26.159s left)


INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_9.ckpt


testing  40% [========            ] (1m 49.919s left)


INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_9.ckpt


testing  60% [============        ] (1m 13.98s left)


INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_9.ckpt


testing  80% [================    ] (36.875s left)


INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/convnet/output/convnet_9.ckpt


testing 100% [====================] (total duration: 3m 3.979s, mean duration: 36.796s)


# add the ranking

In [23]:
# check the ranking before adding to the db for evaluation
checkRankings(rankings,evalData['candidates'],maxUsers=None)

In [24]:
# add the ranking into de DB
addRanking(modelName, rankings, config, logger=logger)